In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-26"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12367,2024-11-26,Eua Nba,00:00,Golden State Warriors,Brooklyn Nets,1.13,6.54,222.5,1.81,2.04,-13.5,2.02,1.83,Iqob9Vqf,0.884956,0.152905,0.552486,0.490196,0.037861,0.4,0.2,NaN,NaN,216.832,119.314692,0.550263,387.504,150.546311,0.388503,184.592,535.834,156.00,408.24,0.0,0.0,0.0,0.0,0.997509,0.084485,0.069792,1.78,0.356,0.365169,0.635958,0.7,0.064042,4.67,0.934,5.931478,0.306609,0.2,-0.106609
12368,2024-11-26,Eua Nba,00:00,Sacramento Kings,Oklahoma City Thunder,2.41,1.61,226.5,1.84,2.00,3.5,2.01,1.86,jcq679E6,0.414938,0.621118,0.543478,0.500000,0.036056,0.2,0.4,NaN,NaN,157.876,17.984740,0.113917,139.642,27.866518,0.199557,150.620,154.288,133.90,136.24,0.0,0.0,0.0,0.0,0.281436,0.058926,0.054814,-2.51,-0.502,-2.808765,0.694181,0.4,-0.294181,-1.10,-0.220,-2.772727,0.642109,0.7,0.057891
12369,2024-11-26,Eua Nba,21:00,Washington Wizards,Chicago Bulls,2.48,1.58,240.5,1.92,1.94,2.5,2.04,1.72,GtEJjvjd,0.403226,0.632911,0.520833,0.515464,0.036137,0.6,0.6,NaN,NaN,579.714,165.048770,0.284707,378.124,63.119008,0.166927,468.482,368.772,699.84,381.60,1.0,0.0,0.0,0.0,0.313496,0.007328,0.120359,-2.49,-0.498,-2.971888,0.310241,0.4,0.089759,0.87,0.174,3.333333,0.332350,0.5,0.167650
12370,2024-11-26,Eua Nba,21:30,Miami Heat,Milwaukee Bucks,1.72,2.20,221.5,1.88,1.96,-3.5,1.99,1.76,dxi6Esr3,0.581395,0.454545,0.531915,0.510204,0.035941,0.6,0.2,NaN,NaN,269.698,65.152205,0.241575,163.468,24.730592,0.151287,181.784,288.184,204.18,297.98,0.0,0.0,0.0,0.0,0.173169,0.029463,0.086738,-2.07,-0.414,-1.739130,0.533069,0.6,0.066931,-5.00,-1.000,-1.200000,0.469748,0.1,-0.369748
12371,2024-11-26,Eua Nba,22:00,Minnesota Timberwolves,Houston Rockets,1.67,2.29,220.5,1.92,1.94,-3.5,1.98,1.82,rgkECLCF,0.598802,0.436681,0.520833,0.515464,0.035484,0.6,0.2,NaN,NaN,179.726,112.370480,0.625232,144.926,32.048458,0.221137,154.358,250.126,123.14,149.48,0.0,0.0,0.0,0.0,0.221417,0.007328,0.059546,-1.06,-0.212,-3.160377,0.690412,0.6,-0.090412,0.67,0.134,9.626866,0.546805,0.6,0.053195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12436,2024-11-26,Rússia Premier League Feminina,09:00,Enisey Krasnoyarsk F,Novosibirsk F,1.30,3.34,137.5,1.85,1.85,-9.5,2.05,1.59,jgLbavXT,0.769231,0.299401,0.540541,0.540541,0.068632,0.6,1.0,NaN,NaN,224.532,151.866677,0.676370,273.894,242.036372,0.883686,193.268,516.996,450.79,286.96,1.0,0.0,0.0,0.0,0.621766,0.000000,0.178719,0.08,0.016,18.750000,0.000000,0.0,0.000000,3.90,0.780,3.000000,0.000000,0.0,0.000000
12437,2024-11-26,Rússia Premier League Feminina,11:00,Nadezhda F,Dynamo Moscow F,1.04,9.10,135.5,1.85,1.85,-17.5,2.05,1.59,4xNj102H,0.961538,0.109890,0.540541,0.540541,0.071429,0.4,0.2,NaN,NaN,256.050,243.999282,0.952936,370.702,464.765894,1.253745,235.622,703.658,676.20,212.31,1.0,0.0,0.0,0.0,1.124118,0.000000,0.178719,1.98,0.396,0.101010,0.000000,0.0,0.000000,-5.00,-1.000,-8.100000,0.000000,0.0,0.000000
12438,2024-11-26,Rússia Premier League Feminina,11:00,UMMC Ekaterinburg F,Dynamo Kursk F,1.30,3.34,143.5,1.85,1.85,-9.5,2.03,1.60,zcVZ4r3h,0.769231,0.299401,0.540541,0.540541,0.068632,0.2,0.0,NaN,NaN,89.860,8.669048,0.096473,520.446,973.601362,1.870706,86.090,0.000,103.00,2262.00,0.0,1.0,0.0,0.0,0.621766,0.000000,0.167524,0.42,0.084,3.571429,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12439,2024-11-2

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Golden State Warriors,Brooklyn Nets,222.5,1.81,0.8993,Over
1,00:00,Eua Nba,Sacramento Kings,Oklahoma City Thunder,226.5,1.84,0.9702,Over
2,15:30,Eua Ncaa,San Francisco,Fordham,142.5,1.91,0.8659,Over
3,21:00,Eua Ncaa,Georgetown,Wagner,128.5,1.95,0.5351,Over
4,22:00,Eua Ncaa,South Alabama,Western Illinois,131.5,1.91,0.8663,Over
5,22:30,Eua Ncaa,Long Beach State,UTEP,133.5,1.91,0.8553,Over
6,23:00,Eua Ncaa,Montana State,Abilene Christian,142.5,1.91,0.8504,Over
